In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LinearRegression
from scipy import sparse
from itertools import product

In [3]:
transactions = pd.read_csv('../transactions.csv')
customers_gender = pd.read_csv('../customers_gender_train.csv')
sex_best = pd.read_csv("../task1/compose-xgb0.7-rf0.2-logreg.csv")

In [4]:
#sex_all = pd.concat([customers_gender, sex_best])

In [5]:
cuses_test = list(set(transactions.customer_id.unique().tolist()).difference(customers_gender.customer_id.unique()))
all_cuses = transactions.customer_id.unique()
all_mcc = transactions.mcc_code.unique()

In [6]:
transactions = transactions[transactions.amount < 0].copy()
transactions['day'] = transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [14]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day,month_num,year_num
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN,23,0,0
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN,24,0,0
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN,24,0,0
4,39026145,2 15:33:42,5499,1010,-920.83,NaN,25,0,0
5,39026145,2 15:53:49,5541,1010,-14643.37,NaN,25,0,0


In [10]:
#transactions = pd.merge(transactions, sex_all)

In [11]:
transactions.day += 29 - transactions['day'].max()%30

In [12]:
transactions['month_num'] = (transactions.day) // 30
#transactions['year_num'] = (transactions.day) // 365

transactions['year_num'] = (transactions.month_num) // 12

In [15]:
test_transactions = transactions[transactions.month_num == 15]
train_transactions = transactions  #[transactions.month_num < 15]

In [16]:
train_transactions.tail()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,day,month_num,year_num
6849340,61870738,452 19:33:04,5541,1110,-4491.83,RU570124,475,15,1
6849341,61870738,453 16:03:02,5499,1010,-5176.84,10217113,476,15,1
6849342,61870738,454 10:54:60,5411,1010,-1652.77,022915,477,15,1
6849343,61870738,454 14:23:59,5499,1010,-4687.23,10217113,477,15,1
6849344,61870738,454 16:11:53,5541,1110,-4491.83,RU570124,477,15,1


In [17]:
test_transactions = test_transactions.set_index('customer_id')
test_transactions = test_transactions.loc[cuses_test]
test_transactions = test_transactions.reset_index()

In [18]:
grid = list(product(*[all_cuses, all_mcc, range(7, 16)]))
train_grid = pd.DataFrame(grid, columns = ['customer_id', 'mcc_code', 'month_num'])

In [19]:
test_grid = list(product(*[cuses_test, all_mcc]))       
test_grid = pd.DataFrame(test_grid, columns = ['customer_id', 'mcc_code'])
test_grid['month_num'] = 16

In [21]:
test_grid.head()

,customer_id,mcc_code,month_num
0,55705601,4814,16
1,55705601,6011,16
2,55705601,4829,16
3,55705601,5499,16
4,55705601,5541,16


In [22]:
%%time
test = pd.merge(test_grid,
         test_transactions.groupby(['customer_id', 'year_num','mcc_code'])[['amount']].sum().reset_index(),
         how='left')#.fillna(0)

CPU times: user 210 ms, sys: 1 ms, total: 211 ms
Wall time: 248 ms


In [23]:
test['year_num'] = (test.month_num) // 12

test = test.drop(["amount"], axis=1)
test.head()

,customer_id,mcc_code,month_num,year_num
0,55705601,4814,16,1
1,55705601,6011,16,1
2,55705601,4829,16,1
3,55705601,5499,16,1
4,55705601,5541,16,1


In [24]:
%%time
train = pd.merge(train_grid,
         train_transactions.groupby(['year_num', 'month_num', 'customer_id', 'mcc_code'])[['amount']].sum().reset_index(),
         how='left').fillna(0)

CPU times: user 11.2 s, sys: 2.15 s, total: 13.4 s
Wall time: 13.4 s


In [25]:
train['year_num'] = (train.month_num) // 12

In [26]:
train.head(7)

,customer_id,mcc_code,month_num,year_num,amount
0,39026145,4814,7,0,-20213.26
1,39026145,4814,8,0,-20662.44
2,39026145,4814,9,0,-7860.72
3,39026145,4814,10,0,-56237.76
4,39026145,4814,11,0,-21336.23
5,39026145,4814,12,1,-32565.80
6,39026145,4814,13,1,-32565.81


In [27]:
%%time
for month_shift in range(1, 5):
    train_shift = train.copy()
    train_shift['month_num'] = train_shift['month_num'] + month_shift
    train_shift['year_num'] = (train_shift.month_num) // 12
    train_shift = train_shift.rename(columns={"amount" : 'amount_{0}'.format(month_shift)})  
    train_shift = train_shift[['year_num', 'month_num', 'customer_id', 'mcc_code', 'amount_{0}'.format(month_shift)]]

    train = pd.merge(train, train_shift, 
                                  on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)
    test = pd.merge(test, train_shift, 
                                 on=['year_num', 'month_num', 'customer_id', 'mcc_code'], how='left').fillna(0)

CPU times: user 1min 39s, sys: 21.5 s, total: 2min 1s
Wall time: 2min 1s


In [114]:
null_index = train[train.amount_1 == 0][train.amount_2 == 0].index#[train.amount_3 == 0][train.amount_4 == 0].index

/home/y.romanikhin/.virtualenvs/venv/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [116]:
train = train.drop(null_index)

In [117]:
train = train.reset_index().drop(["index"], axis=1)

In [29]:
test.head()

,customer_id,mcc_code,month_num,year_num,amount_1,amount_2,amount_3,amount_4
0,55705601,4814,16,1,-26951.00,-13475.50,-41549.45,-112981.92
1,55705601,6011,16,1,-190902.84,-132509.02,-496347.38,-384051.60
2,55705601,4829,16,1,-113418.75,-3368.87,-206646.71,0.00
3,55705601,5499,16,1,-1347.55,-2627.72,0.00,-2672.64
4,55705601,5541,16,1,0.00,0.00,0.00,0.00


In [30]:
%%time
hasher = FeatureHasher(n_features=10000, input_type='string')
train_sparse = \
    hasher.fit_transform(train[['mcc_code']].astype(str).as_matrix())

CPU times: user 2min 16s, sys: 1.31 s, total: 2min 17s
Wall time: 2min 17s


In [31]:
%%time
test_sparse = \
    hasher.transform(test[['mcc_code']].astype(str).as_matrix())

CPU times: user 3.09 s, sys: 12 ms, total: 3.1 s
Wall time: 3.09 s


In [32]:
%time
train_sparse = sparse.hstack([train_sparse,
                              np.log(np.abs(train[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                             ])

test_sparse = sparse.hstack([test_sparse,
                             np.log(np.abs(test[['amount_1', 'amount_2', 'amount_3', 'amount_4']]) + 1).as_matrix()
                            ])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


In [33]:
%%time
shift = 1
clf = LinearRegression(n_jobs=22)
clf.fit(train_sparse, np.log(-train['amount'] + shift))

CPU times: user 5.25 s, sys: 2.46 s, total: 7.71 s
Wall time: 7.71 s


In [34]:
%%time
test['volume'] = np.e ** clf.predict(test_sparse) - shift
#test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_c.csv', index=False)

CPU times: user 60 ms, sys: 0 ns, total: 60 ms
Wall time: 58.8 ms


In [35]:
test.head()

,customer_id,mcc_code,month_num,year_num,amount_1,amount_2,amount_3,amount_4,volume
0,55705601,4814,16,1,-26951.00,-13475.50,-41549.45,-112981.92,27046.117991
1,55705601,6011,16,1,-190902.84,-132509.02,-496347.38,-384051.60,486834.276353
2,55705601,4829,16,1,-113418.75,-3368.87,-206646.71,0.00,8954.657623
3,55705601,5499,16,1,-1347.55,-2627.72,0.00,-2672.64,281.705281
4,55705601,5541,16,1,0.00,0.00,0.00,0.00,2.173343


In [36]:
test[test.volume < 0].head()

,customer_id,mcc_code,month_num,year_num,amount_1,amount_2,amount_3,amount_4,volume


In [37]:
#test['volume'].loc[test["volume"] < 0] = 0
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_16_hash_mcc_only_add_amount3_4.csv', index=False)

### xgb

In [92]:
import xgboost as xgb

In [152]:
param = {}
param['max_depth'] = 3
param['booster'] = 'gbtree'
param['objective'] = 'reg:linear'
param['eval_metric'] = 'rmse'
param['eta'] = 0.05
param['subsample'] = 0.7
param['colsample_bytree'] = 0.8
param['colsample_bylevel'] = 0.8

numround = 100

In [96]:
train.amount_1 = train.amount_1.apply(lambda x: np.log(1-x))

In [97]:
train.amount_2 = train.amount_2.apply(lambda x: np.log(1-x))
train.amount_3 = train.amount_3.apply(lambda x: np.log(1-x))

In [98]:
test.amount_1 = test.amount_1.apply(lambda x: np.log(1-x))
test.amount_2 = test.amount_2.apply(lambda x: np.log(1-x))
test.amount_3 = test.amount_3.apply(lambda x: np.log(1-x))

In [132]:
X_train = train[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]
X_test = test[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]

In [136]:
Xdatatrain = xgb.DMatrix(data=X_train, label = np.log(-train['amount'] + 1))
Xdatatest = xgb.DMatrix(data=X_test)

plst = list(param.items())

In [151]:
%%time
xgb.cv(param, Xdatatrain, num_boost_round=1000, nfold=3, metrics="rmse", \
       verbose_eval=1, as_pandas=False, seed=27)

[0]	train-rmse:1.94011+0.000253142	test-rmse:1.94011+0.000483145
[1]	train-rmse:1.89227+0.0119002	test-rmse:1.89228+0.0113567
[2]	train-rmse:1.86802+0.0123896	test-rmse:1.86806+0.0118619
[3]	train-rmse:1.83612+0.0288408	test-rmse:1.83615+0.0283525
[4]	train-rmse:1.78319+0.0267665	test-rmse:1.78325+0.0262959
[5]	train-rmse:1.73402+0.0250776	test-rmse:1.73407+0.0246411
[6]	train-rmse:1.69534+0.0137811	test-rmse:1.69537+0.0134292
[7]	train-rmse:1.66524+0.0209556	test-rmse:1.66528+0.020499
[8]	train-rmse:1.62654+0.0211091	test-rmse:1.62657+0.0206392
[9]	train-rmse:1.59659+0.0271763	test-rmse:1.5966+0.0267039
[10]	train-rmse:1.57076+0.0314217	test-rmse:1.57076+0.0309225
[11]	train-rmse:1.54287+0.0297486	test-rmse:1.54292+0.0292376
[12]	train-rmse:1.51161+0.027394	test-rmse:1.51166+0.0268867
[13]	train-rmse:1.49562+0.0295131	test-rmse:1.49566+0.029038
[14]	train-rmse:1.47233+0.0220193	test-rmse:1.47237+0.0215726
[15]	train-rmse:1.45313+0.0174394	test-rmse:1.45313+0.0170374
[16]	train-rmse:1.

KeyboardInterrupt: 

In [153]:
%%time
bst = xgb.train(plst, Xdatatrain, numround, verbose_eval = 10)

CPU times: user 2h 57s, sys: 6.94 s, total: 2h 1min 4s
Wall time: 5min 42s


In [154]:
y_pred_xgb = bst.predict(Xdatatest)

In [155]:
min(y_pred_xgb), max(y_pred_xgb)

(-0.10811269, 16.312561)

In [156]:
res_xgb = np.e ** bst.predict(Xdatatest) - shift

In [157]:
res_xgb

array([  1.05005371e+03,   7.02642871e+03,   5.57351641e+04, ...,
        -2.17597485e-02,   1.59621239e-03,   1.00523233e-02], dtype=float32)

In [158]:
test['volume'] = res_xgb

In [159]:
test['volume'].loc[test["volume"] < 0] = 0

In [149]:
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_xgb.csv', index=False)

## compose

In [162]:
y_compose = 0.7*(np.e ** clf.predict(test_sparse) - shift) + 0.3*(np.e ** bst.predict(Xdatatest) - shift)

In [163]:
test['volume'] = y_compose

In [164]:
test['volume'].loc[test["volume"] < 0] = 0

In [166]:
test[['customer_id', 'mcc_code', 'volume']].to_csv('baseline_xgb_linreg07.csv', index=False)

## rf

In [160]:
from sklearn.ensemble import RandomForestClassifier

In [161]:
X_train = train[["mcc_code", "month_num", "year_num", "amount_1", "amount_2", "amount_3"]]

In [128]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=2, n_jobs=-1, oob_score=True)
rf.fit(X_train, list(np.log(-train['amount'] + 1)))

ValueError: Unknown label type: array([[ 10.93736148],
       [  9.96820871],
       [  8.4102403 ],
       ..., 
       [  0.        ],
       [  0.        ],
       [  0.        ]])